# Example C

In [1]:
import enda
import pandas as pd
import os

In [2]:
import enda.ml_backends.sklearn_linreg  
from enda.ml_backends.sklearn_linreg import SKLearnLinearRegression
import joblib

In [3]:
all_models = {
    'lin_reg' : SKLearnLinearRegression(),
    'normalized' : enda.models.NormalizedModel(
        normalized_model = SKLearnLinearRegression(),
        target_col = "slp_kw",
        normalization_col = "kva", #"subscribed_power_kva",
        columns_to_normalize = ["num_contracts"] # ["estimated_annual_consumption_kwh", "num_contracts"]
    ),
    'stacking' : enda.models.StackingModel(
        base_models = {
            "linreg1": SKLearnLinearRegression(),
            "linreg2": SKLearnLinearRegression()
        },
        stacking_model = SKLearnLinearRegression()
    ),
    'fallback' : enda.models.ModelWithFallback(
        resilient_column = "tso_forecast_load_mw",
        model_with = LinearRegression(),
        model_without = LinearRegression()
    )
}

# combine : Normalized Stacking Model
m_stacking = enda.models.StackingModel(
    base_models = {
        "linreg1": SKLearnLinearRegression(),
        "linreg2": SKLearnLinearRegression()
    },
    stacking_model = SKLearnLinearRegression()
)
m = enda.models.NormalizedModel(
    normalized_model = m_stacking,
    target_col = "slp_kw",
    normalization_col = "subscribed_power_kva",
    columns_to_normalize = ["estimated_annual_consumption_kwh", "num_contracts"]
)
all_models['norm_stacking'] = m

# combine : Stacking Normalized Models with different normalization variables
m1 = enda.models.NormalizedModel(
    normalized_model = SKLearnLinearRegression(),
    target_col = "slp_kw",
    normalization_col = "subscribed_power_kva",
    columns_to_normalize = ["estimated_annual_consumption_kwh", "num_contracts"]
)
m2 = enda.models.NormalizedModel(
    normalized_model = SKLearnLinearRegression(),
    target_col = "slp_kw",
    normalization_col = "num_contracts",
    columns_to_normalize = ["estimated_annual_consumption_kwh", "subscribed_power_kva"]
)
m = enda.models.StackingModel(
    base_models = {
        "linreg1": m1,
        "linreg2": m2
    },
    stacking_model = SKLearnLinearRegression()
)
all_models['stacked_norm'] = m


# combine like enercoop for SLP
m_stackin_with_rte = enda.models.StackingModel(
    base_models = {
        "linreg1": SKLearnLinearRegression(),
        "linreg2": SKLearnLinearRegression()
    },
    stacking_model = SKLearnLinearRegression()
)

m_norm_with_rte = enda.models.NormalizedModel(
    normalized_model = m_stackin_with_rte,
    target_col = "slp_kw",
    normalization_col = "subscribed_power_kva",
    columns_to_normalize = ["estimated_annual_consumption_kwh", "num_contracts"]
)

m_stackin_without_rte = m_stacking = enda.models.StackingModel(
    base_models = {
        "linreg1": SKLearnLinearRegression(),
        "linreg2": SKLearnLinearRegression()
    },
    stacking_model = SKLearnLinearRegression()
)

m_norm_without_rte = enda.models.NormalizedModel(
    normalized_model = m_stackin_without_rte,
    target_col = "slp_kw",
    normalization_col = "subscribed_power_kva",
    columns_to_normalize = ["estimated_annual_consumption_kwh", "num_contracts"]
)

m = enda.models.ModelWithFallback(
    resilient_column = "tso_load_forecast_mw",
    model_with = m_norm_with_rte,
    model_without = m_norm_without_rte
)

all_models['enercoop'] = m

NameError: name 'LinearRegression' is not defined